# 2025-26 NBA Season Early Season Catchup
This notebook processes missing game data from the start of the 2025-26 season.


## Imports


In [1]:
from bs4 import BeautifulSoup
import datetime
import numpy as np
import pandas as pd
from time import sleep
from urllib.request import urlopen
import urllib3

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


## Helper Functions


In [2]:
def get_game_suffix(date, team1, team2):
    """Get the game suffix URL for a specific game"""
    http = urllib3.PoolManager()
    r = http.request("GET", f'https://www.basketball-reference.com/boxscores/index.fcgi?year={date.year}&month={date.month}&day={date.day}')
    suffix = None
    if r.status==200:
        soup = BeautifulSoup(r.data, 'html.parser')
        for table in soup.find_all('table', attrs={'class': 'teams'}):
            for anchor in table.find_all('a'):
                if 'boxscores' in anchor.attrs['href']:
                    if team1 in anchor.attrs['href'] or team2 in anchor.attrs['href']:
                        suffix = anchor.attrs['href']
    return suffix


In [3]:
def get_pbp_helper(suffix):
    """Extract play-by-play data from basketball-reference"""
    http = urllib3.PoolManager()
    r = http.request("GET", f'https://www.basketball-reference.com/boxscores/pbp{suffix}')
    if r.status==200:
        soup = BeautifulSoup(r.data, 'html.parser')
        table = soup.find('table', attrs={'id': 'pbp'})
        return pd.read_html(str(table), flavor='lxml')[0]


In [4]:
def format_df(df1):
    """Format the play-by-play dataframe"""
    df1.columns = list(map(lambda x: x[1], list(df1.columns)))
    t1 = list(df1.columns)[1].upper()
    t2 = list(df1.columns)[5].upper()
    q = 1
    df = None
    for index, row in df1.iterrows():
        d = {'QUARTER': float('nan'), 'TIME_REMAINING': float('nan'), f'{t1}_ACTION': float('nan'), f'{t2}_ACTION': float('nan'), f'{t1}_SCORE': float('nan'), f'{t2}_SCORE': float('nan')}
        if row['Time']=='2nd Q':
            q = 2
        elif row['Time']=='3rd Q':
            q = 3
        elif row['Time']=='4th Q':
            q = 4
        elif 'OT' in row['Time']:
            q = row['Time'][0]+'OT'
        try:
            d['QUARTER'] = q
            d['TIME_REMAINING'] = row['Time']
            scores = row['Score'].split('-')
            d[f'{t1}_SCORE'] = int(scores[0])
            d[f'{t2}_SCORE'] = int(scores[1])
            d[f'{t1}_ACTION'] = row[list(df1.columns)[1]]
            d[f'{t2}_ACTION'] = row[list(df1.columns)[5]]
            if df is None:
                df = pd.DataFrame(columns = list(d.keys()))
            df = pd.concat([df, pd.DataFrame(d, index=[0])], ignore_index=True)
        except:
            continue
    return df


In [5]:
def get_pbp(date, team1, team2):
    """Get play-by-play data for a specific game"""
    date = pd.to_datetime(date)
    suffix = get_game_suffix(date, team1, team2).replace('/boxscores', '')
    df = get_pbp_helper(suffix)
    df = df.iloc[1:].reset_index(drop=True)
    df = format_df(df)
    return df


In [6]:
def calculate_game_wp(pbp_df, date, away_team, home_team):
    """Calculate winning time percentage for a game"""
    pbp_df = pbp_df.rename(columns={
        pbp_df.columns[4]:"AWAY_SCORE",
        pbp_df.columns[5]:"HOME_SCORE"
    })

    pbp_df = pbp_df[['QUARTER','TIME_REMAINING','AWAY_SCORE','HOME_SCORE']]
    pbp_df['TIME_REMAINING'] = pbp_df.TIME_REMAINING.replace('24:00.0','12:00.0')
    pbp_df['TIME_REMAINING'] = pd.to_datetime(pbp_df['TIME_REMAINING'])
    pbp_df['TIME_REMAINING'] = pd.to_datetime(pbp_df['TIME_REMAINING'],format= '%H:%M:%S').dt.time
    pbp_df['AWAY_SCORE'] = pbp_df['AWAY_SCORE'].astype(int)
    pbp_df['HOME_SCORE'] = pbp_df['HOME_SCORE'].astype(int)

    quarter_beginning = '12:00:00'
    quarter_ending = '00:00:00'
    overtime_beginning = '05:00:00'

    list_of_quarters = list(pbp_df['QUARTER'].unique())

    full_df = pd.DataFrame()
    for quarter in list_of_quarters:
        quarter_df = pbp_df[pbp_df['QUARTER'] == quarter]

        if quarter == 1:# add beginning of 1st quarter
            quarter_df.loc[-1] = [1, quarter_beginning, 0, 0] 
            quarter_df.index = quarter_df.index + 1  
            quarter_df = quarter_df.sort_index()

            end_row = {
                'QUARTER': [quarter],
                'TIME_REMAINING': quarter_ending,
                'AWAY_SCORE': [quarter_df.iloc[-1,2]],
                'HOME_SCORE': [quarter_df.iloc[-1,3]]
            }

            end_row = pd.DataFrame(data=end_row)
            quarter_df = pd.concat([quarter_df, end_row], axis=0).reset_index(drop=True)

            temp_date = str(datetime.datetime.strptime('1900-01-01', '%Y-%m-%d').date())
            quarter_df['TIME_REMAINING'] = pd.to_datetime(temp_date + " " + quarter_df.TIME_REMAINING.astype(str))
            quarter_df['TIME_ELAPSED'] = (quarter_df['TIME_REMAINING'] - quarter_df['TIME_REMAINING'].shift(-1))
            quarter_df['INDICATOR'] = np.where(
                quarter_df['QUARTER'] == quarter_df['QUARTER'].shift(-1),
                ((quarter_df['TIME_REMAINING'] - quarter_df['TIME_REMAINING'].shift(-1))/60000000000).view(int),
                0
            )

            full_df = pd.concat([full_df, quarter_df], axis=0)


        elif quarter in (2,3,4):
            start_row = end_row
            start_row['QUARTER'] = quarter
            start_row['TIME_REMAINING'] = quarter_beginning

            quarter_df = quarter_df.reset_index(drop=True)
            quarter_df.index += 1

            quarter_df = pd.concat([start_row, quarter_df],axis=0)
            quarter_df = quarter_df.sort_index()

            end_row = {
                'QUARTER': [quarter],
                'TIME_REMAINING': quarter_ending,
                'AWAY_SCORE': [quarter_df.iloc[-1,2]],
                'HOME_SCORE': [quarter_df.iloc[-1,3]]
            }

            end_row = pd.DataFrame(data=end_row)
            quarter_df = pd.concat([quarter_df, end_row], axis=0).reset_index(drop=True)

            temp_date = str(datetime.datetime.strptime('1900-01-01', '%Y-%m-%d').date())
            quarter_df['TIME_REMAINING'] = pd.to_datetime(temp_date + " " + quarter_df.TIME_REMAINING.astype(str))
            quarter_df['TIME_ELAPSED'] = (quarter_df['TIME_REMAINING'] - quarter_df['TIME_REMAINING'].shift(-1))
            quarter_df['INDICATOR'] = np.where(
                quarter_df['QUARTER'] == quarter_df['QUARTER'].shift(-1),
                ((quarter_df['TIME_REMAINING'] - quarter_df['TIME_REMAINING'].shift(-1))/60000000000).view(int),
                0
            )

            full_df = pd.concat([full_df, quarter_df], axis=0)

        else:
            start_row = end_row
            start_row['QUARTER'] = quarter
            start_row['TIME_REMAINING'] = overtime_beginning

            quarter_df = quarter_df.reset_index(drop=True)
            quarter_df.index += 1

            quarter_df = pd.concat([start_row, quarter_df],axis=0)
            quarter_df = quarter_df.sort_index()

            end_row = {
                'QUARTER': [quarter],
                'TIME_REMAINING': quarter_ending,
                'AWAY_SCORE': [quarter_df.iloc[-1,2]],
                'HOME_SCORE': [quarter_df.iloc[-1,3]]
            }

            end_row = pd.DataFrame(data=end_row)
            quarter_df = pd.concat([quarter_df, end_row], axis=0).reset_index(drop=True)

            temp_date = str(datetime.datetime.strptime('1900-01-01', '%Y-%m-%d').date())
            quarter_df['TIME_REMAINING'] = pd.to_datetime(temp_date + " " + quarter_df.TIME_REMAINING.astype(str))
            quarter_df['TIME_ELAPSED'] = (quarter_df['TIME_REMAINING'] - quarter_df['TIME_REMAINING'].shift(-1))
            quarter_df['INDICATOR'] = np.where(
                quarter_df['QUARTER'] == quarter_df['QUARTER'].shift(-1),
                ((quarter_df['TIME_REMAINING'] - quarter_df['TIME_REMAINING'].shift(-1))/60000000000).view(int),
                0
            )

            full_df = pd.concat([full_df, quarter_df], axis=0)

    full_df['LEADER'] = np.where(
        full_df.HOME_SCORE > full_df.AWAY_SCORE,
        'HOME_TEAM',
        'AWAY_TEAM',
    )

    full_df['LEADER'] = np.where(
        full_df.HOME_SCORE == full_df.AWAY_SCORE,
        'NEITHER',
        full_df.LEADER
    )
    full_df_gb = full_df.groupby('LEADER')['INDICATOR'].sum().reset_index() 

    teams = ['AWAY_TEAM','HOME_TEAM']
    total_seconds = full_df_gb.INDICATOR.sum()

    results_row = pd.DataFrame(columns=['DATE','AWAY_TEAM','HOME_TEAM','AWAY_WT','HOME_WT'],index=[0])
    results_row['DATE'] = date
    results_row['AWAY_TEAM'] = away_team
    results_row['HOME_TEAM'] = home_team
    
    try:
        away_seconds_leading = full_df_gb[full_df_gb.LEADER == 'AWAY_TEAM'].INDICATOR.values[0]
    except:
        away_seconds_leading = 0
    away_wp = away_seconds_leading / total_seconds

    try:
        home_seconds_leading = full_df_gb[full_df_gb.LEADER == 'HOME_TEAM'].INDICATOR.values[0]
    except:
        home_seconds_leading = 0
    home_wp = home_seconds_leading / total_seconds
    
    results_row['AWAY_WT'] = away_wp
    results_row['HOME_WT'] = home_wp

    return results_row


In [7]:
def calculate_tm_avg_metric(df, metric):
    """Calculate team average for each metric"""
    # set metric columns
    if metric in ('WT','LT'):
        away_col = 'AWAY_' + metric
        home_col = 'HOME_' + metric
    elif metric == 'TIE_PC':
        away_col = metric
        home_col = metric
    
    # reformat into single column
    nba_results_away = df[['DATE','AWAY_TEAM', away_col]].rename(columns={'AWAY_TEAM':'TEAM',away_col:metric})
    nba_results_home = df[['DATE','HOME_TEAM', home_col]].rename(columns={'HOME_TEAM':'TEAM',home_col:metric})
    
    # concatenate
    nba_results_reformat = pd.concat([nba_results_away, nba_results_home]).reset_index(drop=True)
    
    # find team averages
    nba_results_agg = nba_results_reformat.groupby(['TEAM']).mean().reset_index()
    return nba_results_agg


## Load Existing Data


In [8]:
# Read existing files
nba_wt_results_df = pd.read_csv('25_26_earlyseason_catchup/25_26_wt_results.csv')
nba_wt_results_df['DATE'] = pd.to_datetime(nba_wt_results_df['DATE'])

print(f"Total games in file: {len(nba_wt_results_df)} games")
print(f"\nDate range: {nba_wt_results_df['DATE'].min()} to {nba_wt_results_df['DATE'].max()}")
print(f"\nColumn names: {list(nba_wt_results_df.columns)}")
print(f"\nFirst few rows:")
print(nba_wt_results_df.head())


Total games in file: 1216 games

Date range: 2025-10-21 00:00:00 to 2026-04-12 00:00:00

Column names: ['DATE', 'AWAY_TEAM', 'HOME_TEAM', 'AWAY_WT', 'HOME_WT']

First few rows:
        DATE AWAY_TEAM HOME_TEAM  AWAY_WT  HOME_WT
0 2025-10-21       HOU       OKC      NaN      NaN
1 2025-10-21       GSW       LAL      NaN      NaN
2 2025-10-22       BRK       CHO      NaN      NaN
3 2025-10-22       CLE       NYK      NaN      NaN
4 2025-10-22       MIA       ORL      NaN      NaN


## Identify Games with Missing Data


In [9]:
# Find games where AWAY_WT or HOME_WT are null/empty
# Only look at games before today
today = pd.Timestamp.today().normalize()

# Filter for games before today with missing WT data
missing_data_games = nba_wt_results_df[
    (nba_wt_results_df['DATE'] < today) & 
    (nba_wt_results_df['AWAY_WT'].isna() | nba_wt_results_df['HOME_WT'].isna())
].copy()

print(f"\nGames with missing WT data: {len(missing_data_games)}")
if len(missing_data_games) > 0:
    print(f"Date range: {missing_data_games['DATE'].min()} to {missing_data_games['DATE'].max()}")
    print(f"\nFirst few games with missing data:")
    print(missing_data_games[['DATE', 'AWAY_TEAM', 'HOME_TEAM', 'AWAY_WT', 'HOME_WT']].head(10))
else:
    print("No missing data found!")



Games with missing WT data: 58
Date range: 2025-10-21 00:00:00 to 2025-10-28 00:00:00

First few games with missing data:
        DATE AWAY_TEAM HOME_TEAM  AWAY_WT  HOME_WT
0 2025-10-21       HOU       OKC      NaN      NaN
1 2025-10-21       GSW       LAL      NaN      NaN
2 2025-10-22       BRK       CHO      NaN      NaN
3 2025-10-22       CLE       NYK      NaN      NaN
4 2025-10-22       MIA       ORL      NaN      NaN
5 2025-10-22       PHI       BOS      NaN      NaN
6 2025-10-22       TOR       ATL      NaN      NaN
7 2025-10-22       DET       CHI      NaN      NaN
8 2025-10-22       NOP       MEM      NaN      NaN
9 2025-10-22       WAS       MIL      NaN      NaN


## Process Games with Missing Data
**Note:** This will scrape basketball-reference.com for each game with missing data. 
- Includes automatic rate limiting (sleeps every 9 games for 2 minutes)
- Progress is tracked for each game


In [10]:
# Sort games by date to process chronologically
missing_data_games_sorted = missing_data_games.sort_values('DATE').reset_index(drop=True)

# Dictionary to store updated data
updated_games = {}
failed_games = []

print(f"Starting to process {len(missing_data_games_sorted)} games with missing data...\n")

for idx in range(len(missing_data_games_sorted)):
    # Rate limiting: sleep every 9 games
    if (idx % 5) == 0 and idx > 0:
        print(f"\n[Rate limiting] Sleeping for 120 seconds...\n")
        sleep(120)
    
    date = missing_data_games_sorted.loc[idx, "DATE"]
    away_team = missing_data_games_sorted.loc[idx, "AWAY_TEAM"]
    home_team = missing_data_games_sorted.loc[idx, "HOME_TEAM"]
    
    try:
        print(f"[{idx+1}/{len(missing_data_games_sorted)}] Processing: {date.strftime('%Y-%m-%d')} - {away_team} @ {home_team}")
        
        pbp_df = get_pbp(date, away_team, home_team)
        wp_results = calculate_game_wp(pbp_df, date, away_team, home_team)
        
        # Store the results
        game_key = (pd.Timestamp(date), away_team, home_team)
        updated_games[game_key] = {
            'AWAY_WT': wp_results['AWAY_WT'].values[0],
            'HOME_WT': wp_results['HOME_WT'].values[0]
        }
        
        print(f"  ✓ Success - WT: Away={wp_results['AWAY_WT'].values[0]:.3f}, Home={wp_results['HOME_WT'].values[0]:.3f}")
        
    except Exception as e:
        print(f"  ✗ Failed: {str(e)}")
        failed_games.append({
            'DATE': date,
            'AWAY_TEAM': away_team,
            'HOME_TEAM': home_team,
            'ERROR': str(e)
        })

print(f"\n{'='*60}")
print(f"Processing complete!")
print(f"Successfully processed: {len(updated_games)} games")
print(f"Failed: {len(failed_games)} games")

if len(failed_games) > 0:
    print(f"\nFailed games:")
    failed_df = pd.DataFrame(failed_games)
    print(failed_df)


Starting to process 58 games with missing data...

[1/58] Processing: 2025-10-21 - HOU @ OKC


/var/folders/45/3930d72x3djbt4qzksybmx5r0000gp/T/ipykernel_37438/398105135.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(table), flavor='lxml')[0]
/var/folders/45/3930d72x3djbt4qzksybmx5r0000gp/T/ipykernel_37438/875759817.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pbp_df['TIME_REMAINING'] = pd.to_datetime(pbp_df['TIME_REMAINING'])
/var/folders/45/3930d72x3djbt4qzksybmx5r0000gp/T/ipykernel_37438/875759817.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quarter_df.loc[-1] = [1, quarter_beginning, 0, 

  ✓ Success - WT: Away=0.712, Home=0.178
[2/58] Processing: 2025-10-21 - GSW @ LAL


/var/folders/45/3930d72x3djbt4qzksybmx5r0000gp/T/ipykernel_37438/398105135.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(table), flavor='lxml')[0]
/var/folders/45/3930d72x3djbt4qzksybmx5r0000gp/T/ipykernel_37438/875759817.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pbp_df['TIME_REMAINING'] = pd.to_datetime(pbp_df['TIME_REMAINING'])
/var/folders/45/3930d72x3djbt4qzksybmx5r0000gp/T/ipykernel_37438/875759817.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quarter_df.loc[-1] = [1, quarter_beginning, 0, 

  ✓ Success - WT: Away=0.923, Home=0.040
[3/58] Processing: 2025-10-22 - BRK @ CHO


...

/var/folders/45/3930d72x3djbt4qzksybmx5r0000gp/T/ipykernel_37438/398105135.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(table), flavor='lxml')[0]
/var/folders/45/3930d72x3djbt4qzksybmx5r0000gp/T/ipykernel_37438/875759817.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pbp_df['TIME_REMAINING'] = pd.to_datetime(pbp_df['TIME_REMAINING'])
/var/folders/45/3930d72x3djbt4qzksybmx5r0000gp/T/ipykernel_37438/875759817.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quarter_df.loc[-1] = [1, quarter_beginning, 0, 

  ✓ Success - WT: Away=0.170, Home=0.800
[57/58] Processing: 2025-10-28 - NYK @ MIL


/var/folders/45/3930d72x3djbt4qzksybmx5r0000gp/T/ipykernel_37438/398105135.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(table), flavor='lxml')[0]
/var/folders/45/3930d72x3djbt4qzksybmx5r0000gp/T/ipykernel_37438/875759817.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pbp_df['TIME_REMAINING'] = pd.to_datetime(pbp_df['TIME_REMAINING'])
/var/folders/45/3930d72x3djbt4qzksybmx5r0000gp/T/ipykernel_37438/875759817.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quarter_df.loc[-1] = [1, quarter_beginning, 0, 

  ✓ Success - WT: Away=0.433, Home=0.447
[58/58] Processing: 2025-10-28 - LAC @ GSW
  ✓ Success - WT: Away=0.101, Home=0.873

Processing complete!
Successfully processed: 58 games
Failed: 0 games


/var/folders/45/3930d72x3djbt4qzksybmx5r0000gp/T/ipykernel_37438/398105135.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(table), flavor='lxml')[0]
/var/folders/45/3930d72x3djbt4qzksybmx5r0000gp/T/ipykernel_37438/875759817.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pbp_df['TIME_REMAINING'] = pd.to_datetime(pbp_df['TIME_REMAINING'])
/var/folders/45/3930d72x3djbt4qzksybmx5r0000gp/T/ipykernel_37438/875759817.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quarter_df.loc[-1] = [1, quarter_beginning, 0, 

## Update Original DataFrame with New Data


In [11]:
# Update the original dataframe with the new WT values
for game_key, wt_values in updated_games.items():
    date, away_team, home_team = game_key
    mask = (nba_wt_results_df['DATE'] == date) & \
           (nba_wt_results_df['AWAY_TEAM'] == away_team) & \
           (nba_wt_results_df['HOME_TEAM'] == home_team)
    
    nba_wt_results_df.loc[mask, 'AWAY_WT'] = wt_values['AWAY_WT']
    nba_wt_results_df.loc[mask, 'HOME_WT'] = wt_values['HOME_WT']

# Save updated results
nba_wt_results_df.to_csv('25_26_earlyseason_catchup/25_26_wt_results_updated.csv', index=False)

print(f"Updated results saved!")
print(f"Total games in file: {len(nba_wt_results_df)}")
print(f"Games with data: {nba_wt_results_df['AWAY_WT'].notna().sum()}")
print(f"Games still missing data: {nba_wt_results_df['AWAY_WT'].isna().sum()}")


Updated results saved!
Total games in file: 1216
Games with data: 74
Games still missing data: 1142


## Generate Aggregated Results with Cleaning The Glass Data


In [12]:
# Only use games that have WT data for aggregation
nba_wt_results_complete = nba_wt_results_df[nba_wt_results_df['AWAY_WT'].notna()].copy()

# Calculate LT% and TIE%
nba_wt_results_complete['AWAY_LT'] = nba_wt_results_complete['HOME_WT']
nba_wt_results_complete['HOME_LT'] = nba_wt_results_complete['AWAY_WT']
nba_wt_results_complete['TIE_PC'] = 1 - (nba_wt_results_complete['AWAY_WT'] + nba_wt_results_complete['HOME_WT'])

# Create dataset of wt/lt/tie%
wt_results = calculate_tm_avg_metric(nba_wt_results_complete, 'WT')
lt_results = calculate_tm_avg_metric(nba_wt_results_complete, 'LT')
tie_results = calculate_tm_avg_metric(nba_wt_results_complete, 'TIE_PC')

del wt_results['DATE']
del lt_results['DATE']
del tie_results['DATE']

nba_results_agg = pd.merge(wt_results, lt_results, how='inner', on='TEAM')
nba_results_agg = pd.merge(nba_results_agg, tie_results, how='inner', on='TEAM')

print("Calculated WT, LT, and TIE metrics for all teams")
print(f"\nTeams in aggregation: {len(nba_results_agg)}")
print(f"Based on {len(nba_wt_results_complete)} completed games")


Calculated WT, LT, and TIE metrics for all teams

Teams in aggregation: 30
Based on 74 completed games


In [13]:
# Scrape Cleaning The Glass data
print("Scraping Cleaning The Glass for current standings...")

html = urlopen('https://cleaningtheglass.com/stats/league/summary')
soup = BeautifulSoup(html, 'html.parser')
table = soup.find("table")
df_ctg = pd.read_html(str(table), flavor="lxml")[0]
df_ctg.columns = df_ctg.columns.get_level_values(1)
df_ctg = df_ctg.iloc[: , 1:]
df_ctg = df_ctg.iloc[: , :7]
df_ctg_1 = df_ctg['Team']
df_ctg_2 = df_ctg.iloc[: , 2:5]
df_ctg_3 = df_ctg.iloc[: , 5:]
df_ctg = pd.concat([df_ctg_1, df_ctg_2, df_ctg_3], axis=1)

# Team name mapping
team_names = {
    'full_name':[
        'Atlanta',
        'Boston',
        'Brooklyn',
        'Charlotte',
        'Chicago',
        'Cleveland',
        'Dallas',
        'Denver',
        'Detroit',
        'Golden State',
        'Houston',
        'Indiana',
        'LA Clippers',
        'LA Lakers',
        'Memphis',
        'Miami',
        'Milwaukee',
        'Minnesota',
        'New Orleans',
        'New York',
        'Oklahoma City',
        'Orlando',
        'Philadelphia',
        'Phoenix',
        'Portland',
        'Sacramento',
        'San Antonio',
        'Toronto',
        'Utah',
        'Washington'
    ],
    'abbr_name':[
        'ATL',
        'BOS',
        'BRK',
        'CHO',
        'CHI',
        'CLE',
        'DAL',
        'DEN',
        'DET',
        'GSW',
        'HOU',
        'IND',
        'LAC',
        'LAL',
        'MEM',
        'MIA',
        'MIL',
        'MIN',
        'NOP',
        'NYK',
        'OKC',
        'ORL',
        'PHI',
        'PHO',
        'POR',
        'SAC',
        'SAS',
        'TOR',
        'UTA',
        'WAS'
    ]
}

team_name_map = pd.DataFrame(team_names)

df_ctg = pd.merge(df_ctg, team_name_map, how='inner', left_on='Team', right_on='full_name')
del df_ctg['Team']
del df_ctg['full_name']
df_ctg = df_ctg.rename(columns={
    'abbr_name':'TEAM',
    'Win%':'WP',
    'Exp W82':'EXPECTED_WIN',
    'Point Diff':'PT_DIFF',
    'W':'Wins',
    'L':'Losses'
})

df_ctg['WP'] = df_ctg['WP'].str.rstrip('%').astype('float') / 100.0
df_ctg['EXPECTED_WP'] = df_ctg['EXPECTED_WIN'] / 82

print("Successfully scraped Cleaning The Glass data")


Scraping Cleaning The Glass for current standings...
Successfully scraped Cleaning The Glass data


/var/folders/45/3930d72x3djbt4qzksybmx5r0000gp/T/ipykernel_37438/3269170056.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_ctg = pd.read_html(str(table), flavor="lxml")[0]


In [14]:
# Merge everything together
nba_wt_results_agg = pd.merge(nba_results_agg, df_ctg, how='inner', on='TEAM')
nba_wt_results_agg['WT_v_WP'] = nba_wt_results_agg['WT'] - nba_wt_results_agg['WP']
nba_wt_results_agg['WT_v_EXP_WP'] = nba_wt_results_agg['WT'] - nba_wt_results_agg['EXPECTED_WP']

# Save aggregated results
nba_wt_results_agg.to_csv('25_26_earlyseason_catchup/nba_wt_results_agg_updated.csv', index=False)

print("Aggregated results saved!")
print(f"\nFinal dataset includes {len(nba_wt_results_agg)} teams")
print("\nPreview:")
nba_wt_results_agg.head(10)


Aggregated results saved!

Final dataset includes 30 teams

Preview:


,TEAM,WT,LT,TIE_PC,PT_DIFF,Wins,Losses,WP,EXPECTED_WIN,EXPECTED_WP,WT_v_WP,WT_v_EXP_WP
0,ATL,0.285972,0.650069,0.063958,-8.6,2,3,0.400,21.4,0.260976,-0.114028,0.024997
1,BOS,0.674167,0.287917,0.037917,7.1,2,3,0.400,57.7,0.703659,0.274167,-0.029492
2,BRK,0.070347,0.886042,0.043611,-14.2,0,5,0.000,13.6,0.165854,0.070347,-0.095506
3,CHI,0.574219,0.358854,0.066927,9.0,4,0,1.000,61.9,0.754878,-0.425781,-0.180659
4,CHO,0.432986,0.523194,0.043819,4.1,2,2,0.500,50.7,0.618293,-0.067014,-0.185307
5,CLE,0.696181,0.256875,0.046944,1.8,3,2,0.600,45.5,0.554878,0.096181,0.141303
6,DAL,0.228611,0.727222,0.044167,-7.9,1,3,0.250,21.8,0.265854,-0.021389,-0.037243
7,DEN,0.775324,0.188037,0.036638,9.1,2,1,0.667,60.7,0.740244,0.108324,0.035080
8,DET,0.292917,0.654861,0.052222,0.6,3,2,0.600,42.6,0.519512,-0.307083,-0.226596
9,GSW,0.672480,0.275979,0.051541,7.2,4,1,0.800,57.7,0.703659,-0.127520,-0.031179


## Validation


In [15]:
# Verify that WT + LT + TIE = 1 for all teams
total_check = nba_wt_results_agg['WT'] + nba_wt_results_agg['LT'] + nba_wt_results_agg['TIE_PC']

print("Validation: WT + LT + TIE should equal 1.0")
print(f"Min: {total_check.min():.6f}")
print(f"Max: {total_check.max():.6f}")
print(f"\nAll values valid: {total_check.between(0.999, 1.001).all()}")


Validation: WT + LT + TIE should equal 1.0
Min: 1.000000
Max: 1.000000

All values valid: True


## Summary


In [16]:
print("="*60)
print("CATCHUP COMPLETE")
print("="*60)
print(f"\nFiles updated in: 25_26_earlyseason_catchup/")
print(f"  - 25_26_wt_results.csv (game-by-game results)")
print(f"  - nba_wt_results_agg.csv (team aggregations)")
print(f"\nTotal games in file: {len(nba_wt_results_df)}")
print(f"Games processed/updated: {len(updated_games)}")
print(f"Failed games: {len(failed_games)}")
print(f"\nYou can now use these files with your Lambda function!")


CATCHUP COMPLETE

Files updated in: 25_26_earlyseason_catchup/
  - 25_26_wt_results.csv (game-by-game results)
  - nba_wt_results_agg.csv (team aggregations)

Total games in file: 1216
Games processed/updated: 58
Failed games: 0

You can now use these files with your Lambda function!
